In [9]:
# Homework - Car Fuel Efficiency

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

# Make plots look nice
sns.set()




In [10]:
#Question 1There's one column with missing values. What is it?
#Answer - horsepower
url = "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv"
df = pd.read_csv(url)

df = df[['engine_displacement', 'horsepower', 'vehicle_weight', 'model_year', 'fuel_efficiency_mpg']]

print(df.head())
print(df.isnull().sum())


   engine_displacement  horsepower  vehicle_weight  model_year  \
0                  170       159.0     3413.433759        2003   
1                  130        97.0     3149.664934        2007   
2                  170        78.0     3079.038997        2018   
3                  220         NaN     2542.392402        2009   
4                  210       140.0     3460.870990        2009   

   fuel_efficiency_mpg  
0            13.231729  
1            13.688217  
2            14.246341  
3            16.912736  
4            12.488369  
engine_displacement      0
horsepower             708
vehicle_weight           0
model_year               0
fuel_efficiency_mpg      0
dtype: int64


In [ ]:
import pandas as pd
#Question 2
#What's the median (50% percentile) for variable 'horsepower'?

# Drop missing values
df = df.dropna()

# Calculate median horsepowers
median_hp = df['horsepower'].median()
print("Median horsepower:", median_hp)
#Answer = 149.0


Median horsepower: 149.0


In [20]:
# Question 3: Handling missing values (0 vs mean)

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

# --- load dataset
url = "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv"
df = pd.read_csv(url)
df = df[['engine_displacement', 'horsepower', 'vehicle_weight', 'model_year', 'fuel_efficiency_mpg']]
print("Loaded dataframe shape:", df.shape)
print(df.head())
print("\nMissing values:\n", df.isnull().sum())

# --- shuffle + split (seed 42)
np.random.seed(42)
n = len(df)
n_val = int(0.2 * n)
n_test = int(0.2 * n)
n_train = n - (n_val + n_test)

idx = np.arange(n)
np.random.shuffle(idx)
df_shuffled = df.iloc[idx].reset_index(drop=True)

df_train = df_shuffled.iloc[:n_train].copy().reset_index(drop=True)
df_val   = df_shuffled.iloc[n_train:n_train+n_val].copy().reset_index(drop=True)
df_test  = df_shuffled.iloc[n_train+n_val:].copy().reset_index(drop=True)

print("\nSplit sizes -> train, val, test:", len(df_train), len(df_val), len(df_test))

# --- targets
y_train = df_train['fuel_efficiency_mpg'].values
y_val   = df_val['fuel_efficiency_mpg'].values

# drop target from features
del df_train['fuel_efficiency_mpg']
del df_val['fuel_efficiency_mpg']
del df_test['fuel_efficiency_mpg']

# --- linear regression (closed-form, no regularization)
def train_linear_regression(X, y):
    ones = np.ones(X.shape[0])
    Xb = np.column_stack([ones, X])
    XTX = Xb.T.dot(Xb)
    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(Xb.T).dot(y)
    return w[0], w[1:]

def prepare_X_from_df(df_features):
    return df_features[df_train.columns].values.astype(float)

# --- Option A: fill horsepower with 0
df_train_0 = df_train.copy()
df_val_0   = df_val.copy()
df_train_0['horsepower'] = df_train_0['horsepower'].fillna(0)
df_val_0['horsepower']   = df_val_0['horsepower'].fillna(0)

X_train_0 = prepare_X_from_df(df_train_0)
X_val_0   = prepare_X_from_df(df_val_0)

w0_0, w_0 = train_linear_regression(X_train_0, y_train)
y_val_pred_0 = w0_0 + X_val_0.dot(w_0)
rmse_0 = np.sqrt(mean_squared_error(y_val, y_val_pred_0))

# --- Option B: fill horsepower with mean (computed on train only)
hp_mean = df_train['horsepower'].mean()
print("\nMean horsepower computed from TRAIN:", hp_mean)

df_train_mean = df_train.copy()
df_val_mean   = df_val.copy()
df_train_mean['horsepower'] = df_train_mean['horsepower'].fillna(hp_mean)
df_val_mean['horsepower']   = df_val_mean['horsepower'].fillna(hp_mean)

X_train_mean = prepare_X_from_df(df_train_mean)
X_val_mean   = prepare_X_from_df(df_val_mean)

w0_m, w_m = train_linear_regression(X_train_mean, y_train)
y_val_pred_mean = w0_m + X_val_mean.dot(w_m)
rmse_mean = np.sqrt(mean_squared_error(y_val, y_val_pred_mean))

# --- results ---
print("RMSE (fill with 0)   :", round(rmse_0, 2))
print("RMSE (fill with mean):", round(rmse_mean, 2))

if round(rmse_0, 2) == round(rmse_mean, 2):
    print("Conclusion: Both are equally good")
elif rmse_mean < rmse_0:
    print("Conclusion: Filling with mean gives better RMSE")
else:
    print("Conclusion: Filling with 0 gives better RMSE")


Loaded dataframe shape: (9704, 5)
   engine_displacement  horsepower  vehicle_weight  model_year  \
0                  170       159.0     3413.433759        2003   
1                  130        97.0     3149.664934        2007   
2                  170        78.0     3079.038997        2018   
3                  220         NaN     2542.392402        2009   
4                  210       140.0     3460.870990        2009   

   fuel_efficiency_mpg  
0            13.231729  
1            13.688217  
2            14.246341  
3            16.912736  
4            12.488369  

Missing values:
 engine_displacement      0
horsepower             708
vehicle_weight           0
model_year               0
fuel_efficiency_mpg      0
dtype: int64

Split sizes -> train, val, test: 5824 1940 1940

Mean horsepower computed from TRAIN: 149.54476367006487
RMSE (fill with 0)   : 0.52
RMSE (fill with mean): 0.46
Conclusion: Filling with mean gives better RMSE


In [22]:
#Question 4
import pandas as pd
import numpy as np

# --- Load dataset
url = "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv"
df = pd.read_csv(url)
df = df[['engine_displacement', 'horsepower', 'vehicle_weight', 'model_year', 'fuel_efficiency_mpg']]

# --- Shuffle + split (seed 42)
np.random.seed(42)
n = len(df)
n_val = int(0.2 * n)
n_test = int(0.2 * n)
n_train = n - (n_val + n_test)

idx = np.arange(n)
np.random.shuffle(idx)
df_shuffled = df.iloc[idx].reset_index(drop=True)

df_train = df_shuffled.iloc[:n_train].copy().reset_index(drop=True)
df_val   = df_shuffled.iloc[n_train:n_train+n_val].copy().reset_index(drop=True)
df_test  = df_shuffled.iloc[n_train+n_val:].copy().reset_index(drop=True)

print("Split sizes:", len(df_train), len(df_val), len(df_test))

# --- Fill missing horsepower with 0 (per Q4 requirement)
for dataset in [df_train, df_val, df_test]:
    dataset['horsepower'] = dataset['horsepower'].fillna(0)

# --- RMSE function
def rmse(y, y_pred):
    return np.sqrt(((y - y_pred) ** 2).mean())

# --- Regularized linear regression
def train_linear_regression_reg(X, y, r=0.0):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    reg = r * np.eye(XTX.shape[0])
    XTX = XTX + reg

    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X.T).dot(y)
    
    return w[0], w[1:]

# --- Prepare X
def prepare_X(df):
    df_num = df.drop('fuel_efficiency_mpg', axis=1)
    return df_num.values.astype(float)

# --- Prepare data
y_train = df_train['fuel_efficiency_mpg'].values
y_val   = df_val['fuel_efficiency_mpg'].values
y_test  = df_test['fuel_efficiency_mpg'].values

X_train = prepare_X(df_train)
X_val   = prepare_X(df_val)
X_test  = prepare_X(df_test)

# --- Try different values of r
results = {}
for r in [0, 0.01, 0.1, 1, 5, 10, 100]:
    w0, w = train_linear_regression_reg(X_train, y_train, r=r)
    y_pred = w0 + X_val.dot(w)
    score = rmse(y_val, y_pred)
    results[r] = round(score, 2)
    print(f"r={r} -> RMSE={round(score, 2)}")

# --- Find best r
best_r = min(results, key=lambda k: (results[k], k))
print("\nBest r:", best_r, "with RMSE:", results[best_r])

# --- Final test evaluation
w0, w = train_linear_regression_reg(X_train, y_train, r=best_r)
y_pred_test = w0 + X_test.dot(w)
print("Test RMSE:", round(rmse(y_test, y_pred_test), 2))



Split sizes: 5824 1940 1940
r=0 -> RMSE=0.52
r=0.01 -> RMSE=0.52
r=0.1 -> RMSE=0.52
r=1 -> RMSE=0.52
r=5 -> RMSE=0.52
r=10 -> RMSE=0.52
r=100 -> RMSE=0.52

Best r: 0 with RMSE: 0.52
Test RMSE: 0.52


In [24]:
# Question 5: Effect of different seeds

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

# --- load dataset
url = "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv"
df = pd.read_csv(url)
df = df[['engine_displacement', 'horsepower', 'vehicle_weight', 'model_year', 'fuel_efficiency_mpg']]

# --- function to prepare features (fill NA with 0)
def prepare_X(df):
    df_num = df.copy()
    df_num = df_num.fillna(0)
    return df_num.values

# --- linear regression (no regularization)
def train_linear_regression(X, y):
    ones = np.ones(X.shape[0])
    Xb = np.column_stack([ones, X])
    XTX = Xb.T.dot(Xb)
    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(Xb.T).dot(y)
    return w[0], w[1:]

def rmse(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

# --- try multiple seeds
seeds = [0,1,2,3,4,5,6,7,8,9]
scores = []

for seed in seeds:
    np.random.seed(seed)
    n = len(df)
    n_val = int(0.2 * n)
    n_test = int(0.2 * n)
    n_train = n - (n_val + n_test)

    idx = np.arange(n)
    np.random.shuffle(idx)
    df_shuffled = df.iloc[idx].reset_index(drop=True)

    df_train = df_shuffled.iloc[:n_train].copy()
    df_val   = df_shuffled.iloc[n_train:n_train+n_val].copy()
    df_test  = df_shuffled.iloc[n_train+n_val:].copy()

    y_train = df_train['fuel_efficiency_mpg'].values
    y_val   = df_val['fuel_efficiency_mpg'].values

    del df_train['fuel_efficiency_mpg']
    del df_val['fuel_efficiency_mpg']
    del df_test['fuel_efficiency_mpg']

    X_train = prepare_X(df_train)
    X_val   = prepare_X(df_val)

    w0, w = train_linear_regression(X_train, y_train)
    y_val_pred = w0 + X_val.dot(w)
    score = rmse(y_val, y_val_pred)

    print(f"Seed={seed}, RMSE={score:.3f}")
    scores.append(score)

# --- compute std
std = np.std(scores)
print("\nValidation RMSEs:", [round(s,3) for s in scores])
std_rounded = round(std, 3)

print("Standard Deviation (rounded):", std_rounded)



Seed=0, RMSE=0.521
Seed=1, RMSE=0.521
Seed=2, RMSE=0.523
Seed=3, RMSE=0.516
Seed=4, RMSE=0.511
Seed=5, RMSE=0.528
Seed=6, RMSE=0.531
Seed=7, RMSE=0.509
Seed=8, RMSE=0.515
Seed=9, RMSE=0.513

Validation RMSEs: [np.float64(0.521), np.float64(0.521), np.float64(0.523), np.float64(0.516), np.float64(0.511), np.float64(0.528), np.float64(0.531), np.float64(0.509), np.float64(0.515), np.float64(0.513)]
Standard Deviation (rounded): 0.007


In [28]:
# Question 6:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

# --- Load dataset
url = "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv"
df = pd.read_csv(url)

# Keep only numeric + target column
df = df[['engine_displacement', 'horsepower', 'vehicle_weight', 'model_year', 'fuel_efficiency_mpg']]

# --- Helper functions
def prepare_X(df):
    df_num = df.copy()
    df_num = df_num.fillna(0)
    return df_num.values

def train_linear_regression_reg(X, y, r=0.0):
    ones = np.ones(X.shape[0])
    Xb = np.column_stack([ones, X])

    # Ridge regularization
    XTX = Xb.T.dot(Xb)
    reg = r * np.eye(XTX.shape[0])
    reg[0,0] = 0  # don't regularize bias
    XTX_inv = np.linalg.inv(XTX + reg)

    w = XTX_inv.dot(Xb.T).dot(y)
    return w[0], w[1:]

def rmse(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))


# --- Split dataset with seed=9
np.random.seed(9)

n = len(df)
n_val = int(0.2 * n)
n_test = int(0.2 * n)
n_train = n - (n_val + n_test)

idx = np.arange(n)
np.random.shuffle(idx)

df_shuffled = df.iloc[idx].reset_index(drop=True)

df_train = df_shuffled.iloc[:n_train].copy()
df_val = df_shuffled.iloc[n_train:n_train+n_val].copy()
df_test = df_shuffled.iloc[n_train+n_val:].copy()

# Combine train + val
df_full_train = pd.concat([df_train, df_val])

y_full_train = df_full_train['fuel_efficiency_mpg'].values
y_test = df_test['fuel_efficiency_mpg'].values

del df_full_train['fuel_efficiency_mpg']
del df_test['fuel_efficiency_mpg']

X_full_train = prepare_X(df_full_train)
X_test = prepare_X(df_test)

# --- Train with r=0.001
w0, w = train_linear_regression_reg(X_full_train, y_full_train, r=0.001)

# --- Predict on test
y_pred = w0 + X_test.dot(w)
score = rmse(y_test, y_pred)

print(f"Test RMSE: {score:.3f}")


Test RMSE: 0.516
